# Test 4POC

In [ ]:
import molsysmt as msm
import openmm as mm
from openmm import app
from openmm import unit

#molsys = msm.convert('4POC')
#molsys = msm.extract(molsys, selection='molecule_type=="protein"')
#molsys = msm.build.add_missing_hydrogens(molsys)
#molsys = msm.build.solvate(molsys)

molsys = msm.convert('initial.msmpk')

topology = msm.convert(molsys, 'openmm.Topology')
coordinates = msm.get(molsys, coordinates=True)
positions = msm.pyunitwizard.convert(coordinates[0], to_form='openmm.unit')

forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')


system = forcefield.createSystem(topology, nonbondedMethod=app.PME,
        nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)

integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)

platform = mm.Platform.getPlatformByName("CPU")

simulation = app.Simulation(topology, system, integrator, platform)

simulation.context.setPositions(positions)

#simulation.minimizeEnergy()
#msm.convert(simulation, 'initial.msmpk')

#from molsysmt.thirds.openmm.reporters import MolSysMTTrajectoryDictReporter
from molsysmt.thirds.openmm.reporters import MSMH5Reporter
from mdtraj.reporters import HDF5Reporter
from molsysmt.thirds.openmm.reporters import TQDMReporter

tqdm_reporter = TQDMReporter(100, 5000)
simulation.reporters.append(tqdm_reporter)


#h5_reporter = HDF5Reporter('test.h5', 100)
#simulation.reporters.append(h5_reporter)

#dict_reporter = MolSysMTTrajectoryDictReporter(10)
#simulation.reporters.append(dict_reporter)

msmh5_reporter = MSMH5Reporter('test.msmh5', 100, steps=5000, selection='all',
            topology=topology, time=True, box=True, coordinates=True, velocities=False,
            potentialEnergy=True, kineticEnergy=True, temperature=True,
            includeInitialContext=True, constantReportInterval=True,
            constantStepSize=True, constantBox=True)
simulation.reporters.append(msmh5_reporter)

%%time
simulation.step(5000)

h5_reporter.close()

msmh5_reporter.close()

traj = dict_reporter.finalize()
msmh5_reporter.close()